TO DO:
- Add option to force players into the team, i.e., optimize given certain player are already locked in

In [1]:
# previous gameweek
gameweek=15

In [2]:
import pandas as pd
import numpy as np
import json
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [3]:
# fetch player_dict 
with open('../data/player_name_dict.txt', 'r') as f:
    player_name_dict_as_text = f.read()
    player_name_dict = json.loads(player_name_dict_as_text)

# Solver definition

In [4]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES

    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    captaincy_variables = [LpVariable(name=f'cap_{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'price'] for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'position']=='GK' else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'position']=='DEF' else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'position']=='MID' else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'position']=='FWD' else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # select 1 captain
    model += lpSum(captaincy_variables) == 1
    # captain must be one of the 11 players in the team
    for i in range(0,len(variables)):
        model += captaincy_variables[i] <= variables[i]
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3

    # OBJECTIVE
    # the second part doubles the captain's points
    model += np.dot(expected_points,variables) + np.dot(expected_points,captaincy_variables)

    # SOLVE OPTIMIZATION

    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [str(var) for var in model.variables() if var.value()==1]
    captain = [player for player in players if 'cap' in player]
    captain = int(captain[0].replace('cap_', ''))
    players = [int(player) for player in players if 'cap' not in player]
    dream_team = df.loc[players]
    cost = (dream_team['price']).sum()
    exp_points = dream_team[expected_column].sum() + df.loc[captain, expected_column]
    print(f'Cost: {cost}')
    print(f'Expected points per week: {exp_points}')
    
    display(dream_team[['position','name', 'team', 'price',expected_column]].sort_values('position'))
    
    return players

# Data

In [5]:
#projections = pd.read_csv(f'../point_projections/xP_projections_week{gameweek}.csv', index_col=0)
projections = pd.read_csv(f'../point_projections/collective_xP_projections_week{gameweek}.csv', index_col=0)
projections = projections.rename(columns={'xP':'projected_points'})
projections.head(5)

,name,team,opp_team,was_home,gameweek,projected_points,std
0,Granit Xhaka,Arsenal,Wolves,0,16,3.205358,0.157584
1,Granit Xhaka,Arsenal,West Ham,1,17,3.047157,0.217339
2,Granit Xhaka,Arsenal,Brighton,0,18,2.792313,0.338886
3,Granit Xhaka,Arsenal,Newcastle Utd,1,19,3.028453,0.382243
4,Granit Xhaka,Arsenal,Tottenham,0,20,2.642736,0.101967


In [6]:
# decay for future projected points to model uncertainty increasing over time
decay_coeff = 0.875
projections['projected_points_with_decay'] = ( projections['projected_points']*decay_coeff
                                              **(projections['gameweek']-(gameweek+1)) )
projections.head(5)

,name,team,opp_team,was_home,gameweek,projected_points,std,projected_points_with_decay
0,Granit Xhaka,Arsenal,Wolves,0,16,3.205358,0.157584,3.205358
1,Granit Xhaka,Arsenal,West Ham,1,17,3.047157,0.217339,2.666262
2,Granit Xhaka,Arsenal,Brighton,0,18,2.792313,0.338886,2.137865
3,Granit Xhaka,Arsenal,Newcastle Utd,1,19,3.028453,0.382243,2.028827
4,Granit Xhaka,Arsenal,Tottenham,0,20,2.642736,0.101967,1.549124


In [7]:
fpl_df = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv', index_col=0)
fpl_df['name'] = fpl_df['name'].map(player_name_dict).fillna(fpl_df['name'])
fpl_df.head(5)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,100.0,100.0,58822,0,0,-3,3,0,2,0.7,...,2022-23,Cedric Alves Soares,DEF,15,4.2,10876.416,-130,Chelsea,2022-11-06T12:00:00Z,0
1,NaN,NaN,84450,0,0,1,-1,1,3,5.3,...,2022-23,Granit Xhaka,MID,15,5.1,424180.224,-8825,Chelsea,2022-11-06T12:00:00Z,0
2,100.0,100.0,153256,0,0,-3,3,1,3,0.7,...,2022-23,Mohamed Elneny,MID,15,4.2,76134.912,4786,Chelsea,2022-11-06T12:00:00Z,0
3,NaN,NaN,156074,0,0,-3,3,0,2,1.0,...,2022-23,Rob Holding,DEF,15,4.2,10876.416,-97,Chelsea,2022-11-06T12:00:00Z,0
4,100.0,100.0,167199,0,0,-2,2,0,3,4.7,...,2022-23,Thomas Partey,MID,15,4.8,54382.080,667,Chelsea,2022-11-06T12:00:00Z,0


In [9]:
# choose time horizon for optimization
horizon = 1

# create dataframe to be given as input for optimization
df = projections[projections['gameweek']<=gameweek+horizon].groupby(by=['name']).mean()\
                                                        [['projected_points','projected_points_with_decay', 'std']] 
#df = df.to_frame()
df = df.reset_index()
df = df.rename(columns={'projected_points':'projected_points_per_game'})
#df['team'] = [fpl_df.loc[fpl_df['name']==name, 'team'].values[0] for name in df['name']]
df['price'] = [fpl_df.loc[fpl_df['name']==name, 'value'].values[0] for name in df['name']]
df['position'] = [fpl_df.loc[fpl_df['name']==name, 'element_type'].values[0] for name in df['name']]
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
df['position'] = df['position'].map(position_dict)
df['position'] = pd.Categorical(df['position'], ['GK','DEF','MID','FWD'])
df['team'] = [projections.loc[projections['name']==name, 'team'].head(1).values[0] for name in df['name']]
# re-order columns
df = df[['name', 'team', 'position', 'price', 'projected_points_per_game', 'projected_points_with_decay', 'std']]
df.head(5)

,name,team,position,price,projected_points_per_game,projected_points_with_decay,std
0,Aaron Cresswell,West Ham,DEF,4.8,3.350040,3.350040,0.107745
1,Aaron Hickey,Brentford,DEF,4.9,1.735689,1.735689,0.098689
2,Aaron Ramsdale,Arsenal,GK,4.9,3.717104,3.717104,0.111077
3,Aaron Wan-Bissaka,Manchester Utd,DEF,4.3,2.932375,2.932375,0.202218
4,Abdoulaye Doucoure,Everton,MID,5.3,2.452793,2.452793,0.172072


In [10]:
# edit some player prices according to my selling price
df.loc[df['name']=='Erling Haaland', 'price'] = 11.8
df.loc[df['name']=='Martinelli', 'price'] = 6.3

# Optimization

In [11]:
team_value = 100.7
money_in_the_bank = 2.7
allowed_bench_value = 17.5
max_price = team_value + money_in_the_bank - allowed_bench_value

## Without decay

In [ ]:
players = optimize(df, max_price, 'projected_points_per_game')

In [ ]:
ban = []
# ban some cheap keepers in good teams who don't actually play
cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
                 'Karl Darlow', 'Darren Randolph']
cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
ban = ban + list(cheap_keepers_ix)
# ban given players
ban_field = ['Marcos Alonso', 'Ben Chilwell', 'Riyad Mahrez', 'Darwin Nunez', 'Aymeric Laporte', 'Emil Krafth', 
             'Sergio Gomez', 'Nathaniel Phillips']
ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [ ]:
players = optimize(my_df, max_price, 'projected_points_per_game')

## With decay

In [ ]:
#players = optimize(df, max_price, 'projected_points_with_decay')

In [12]:
ban = []
# ban some cheap keepers in good teams who don't actually play
cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
                 'Karl Darlow', 'Darren Randolph', 'Alphonse Areola', 'Asmir Begovic', 
                'Sam Johnstone', 'Jack Butland']
cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
ban = ban + list(cheap_keepers_ix)
# ban given players
ban_field = ['Marcos Alonso', 'Riyad Mahrez', 'Aymeric Laporte', 'Nathaniel Phillips', 'Kyle Walker', 
             'Alexander Isak']
ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [13]:
players = optimize(my_df, max_price, 'projected_points_with_decay')

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/5333771ddc10424a81f814d6bea93a64-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/5333771ddc10424a81f814d6bea93a64-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 524 COLUMNS
At line 7132 RHS
At line 7652 BOUNDS
At line 8633 ENDATA
Problem MODEL has 519 rows, 980 columns and 3667 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 62.8039 - 0.01 seconds
Cgl0004I processed model has 517 rows, 980 columns (980 integer (980 of which binary)) and 3430 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 1.33227e-15
Cbc0038I Solution found of -62.8039
Cbc0038I Before mini branch and bound, 980 integers at bound fixed and 0 continuous
Cbc0038

,position,name,team,price,projected_points_with_decay
21,GK,Alisson,Liverpool,5.4,4.213799
154,DEF,Gabriel Dos Santos,Arsenal,5.1,4.068051
27,DEF,Andrew Robertson,Liverpool,6.7,4.669917
495,DEF,William Saliba,Arsenal,5.3,4.049405
264,MID,Kevin De Bruyne,Manchester City,12.6,5.850493
323,MID,Martin degaard,Arsenal,6.4,4.873374
341,MID,Miguel Almiron,Newcastle Utd,5.8,4.701961
345,MID,Mohamed Salah,Liverpool,12.7,7.195047
493,MID,Wilfried Zaha,Crystal Palace,7.5,4.787946
14,FWD,Aleksandar Mitrovic,Fulham,6.8,4.941288


# Sensitivity analysis

Here we add small random Gaussian noise to each players' projected points and run the optimization with these new estimates. We repeat the process n times and keep a list of how many times each player appeared in the optimal team.

In [14]:
ban = []
# ban some cheap keepers in good teams who don't actually play
cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
                 'Karl Darlow', 'Darren Randolph', 'Alphonse Areola', 'Kepa Arrizabalaga', 'Asmir Begovic', 
                'Sam Johnstone', 'Jack Butland']
cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
ban = ban + list(cheap_keepers_ix)
# ban given players
ban_field = ['Marcos Alonso', 'Riyad Mahrez', 'Aymeric Laporte', 'Nathaniel Phillips']
ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [15]:
metric = 'projected_points_with_decay'
# set the standard deviation of the Gaussian noise to be a fraction of each players' projected points
#sigma_factor = 0.1
simulation_rounds = 100
results = []
for i in range(0,simulation_rounds):
    df_with_noise = my_df.copy()
    #df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise[metric]*sigma_factor)
    df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise['std'])
    players = optimize(df_with_noise, max_price, metric)
    results.append(players)

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/6723c0fbbc0c455bb917f8c91cee7f27-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/6723c0fbbc0c455bb917f8c91cee7f27-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 525 COLUMNS
At line 7148 RHS
At line 7669 BOUNDS
At line 8652 ENDATA
Problem MODEL has 520 rows, 982 columns and 3676 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 65.9821 - 0.00 seconds
Cgl0004I processed model has 518 rows, 982 columns (982 integer (982 of which binary)) and 3437 elements
Cbc0038I Initial state - 4 integers unsatisfied sum - 1.41935
Cbc0038I Solution found of -65.8944
Cbc0038I Before mini branch and bound, 978 integers at bound fixed and 0 continuous
Cbc0038I Fu

,position,name,team,price,projected_points_with_decay
131,GK,Ederson,Manchester City,5.4,4.273410
135,DEF,Emil Krafth,Newcastle Utd,4.3,4.134355
275,DEF,Kyle Walker,Manchester City,4.8,4.503234
476,DEF,Trent Alexander-Arnold,Liverpool,7.2,4.659084
341,MID,Miguel Almiron,Newcastle Utd,5.8,4.842563
345,MID,Mohamed Salah,Liverpool,12.7,7.484801
493,MID,Wilfried Zaha,Crystal Palace,7.5,4.831085
64,MID,Bukayo Saka,Arsenal,8.0,4.934151
14,FWD,Aleksandar Mitrovic,Fulham,6.8,5.341819
143,FWD,Erling Haaland,Manchester City,11.8,7.336598


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/798b33e9d2dc4d0982d707dc6d5e4eaa-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/798b33e9d2dc4d0982d707dc6d5e4eaa-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 525 COLUMNS
At line 7148 RHS
At line 7669 BOUNDS
At line 8652 ENDATA
Problem MODEL has 520 rows, 982 columns and 3676 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 64.494 - 0.00 seconds
Cgl0004I processed model has 518 rows, 982 columns (982 integer (982 of which binary)) and 3437 elements
Cbc0038I Initial state - 4 integers unsatisfied sum - 1.55556
Cbc0038I Pass   1: suminf.    0.63636 (2) obj. -63.6718 iterations 6
Cbc0038I Solution found of -60.9114
Cbc0038I Before mini branch

,position,name,team,price,projected_points_with_decay
131,GK,Ederson,Manchester City,5.4,4.065366
154,DEF,Gabriel Dos Santos,Arsenal,5.1,4.448091
27,DEF,Andrew Robertson,Liverpool,6.7,4.711376
476,DEF,Trent Alexander-Arnold,Liverpool,7.2,4.633450
323,MID,Martin degaard,Arsenal,6.4,4.973727
324,MID,Martinelli,Arsenal,6.3,4.437097
341,MID,Miguel Almiron,Newcastle Utd,5.8,4.745501
345,MID,Mohamed Salah,Liverpool,12.7,6.725568
14,FWD,Aleksandar Mitrovic,Fulham,6.8,5.061621
143,FWD,Erling Haaland,Manchester City,11.8,7.282203


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/8cf8145cbbb749f9b68f983feac47f64-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/8cf8145cbbb749f9b68f983feac47f64-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 525 COLUMNS
At line 7148 RHS
At line 7669 BOUNDS
At line 8652 ENDATA
Problem MODEL has 520 rows, 982 columns and 3676 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 63.4045 - 0.00 seconds
Cgl0004I processed model has 518 rows, 982 columns (982 integer (982 of which binary)) and 3437 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.484848
Cbc0038I Pass   1: suminf.    0.18391 (2) obj. -63.1904 iterations 9
Cbc0038I Solution found of -59.8939
Cbc0038I Before mini bran

...

,position,name,team,price,projected_points_with_decay
131,GK,Ederson,Manchester City,5.4,4.282616
154,DEF,Gabriel Dos Santos,Arsenal,5.1,4.519068
224,DEF,Joao Cancelo,Manchester City,7.4,4.720303
27,DEF,Andrew Robertson,Liverpool,6.7,4.807114
279,MID,Leandro Trossard,Brighton,7.1,4.923196
324,MID,Martinelli,Arsenal,6.3,4.707065
345,MID,Mohamed Salah,Liverpool,12.7,7.363040
447,MID,Solly March,Brighton,5.0,4.334734
14,FWD,Aleksandar Mitrovic,Fulham,6.8,5.914239
143,FWD,Erling Haaland,Manchester City,11.8,6.132607


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/0f514df12da24a30a65ae9893ab24c81-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/0f514df12da24a30a65ae9893ab24c81-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 525 COLUMNS
At line 7148 RHS
At line 7669 BOUNDS
At line 8652 ENDATA
Problem MODEL has 520 rows, 982 columns and 3676 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 67.1219 - 0.00 seconds
Cgl0004I processed model has 518 rows, 982 columns (982 integer (982 of which binary)) and 3437 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.222222
Cbc0038I Solution found of -67.0174
Cbc0038I Before mini branch and bound, 980 integers at bound fixed and 0 continuous
Cbc0038I F

,position,name,team,price,projected_points_with_decay
21,GK,Alisson,Liverpool,5.4,4.319877
224,DEF,Joao Cancelo,Manchester City,7.4,4.799566
27,DEF,Andrew Robertson,Liverpool,6.7,4.733264
277,DEF,Layvin Kurzawa,Fulham,4.3,4.551331
264,MID,Kevin De Bruyne,Manchester City,12.6,5.984366
301,MID,Luke Harris,Fulham,4.4,6.373309
323,MID,Martin degaard,Arsenal,6.4,4.724499
345,MID,Mohamed Salah,Liverpool,12.7,7.829951
493,MID,Wilfried Zaha,Crystal Palace,7.5,4.867053
14,FWD,Aleksandar Mitrovic,Fulham,6.8,4.847266


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/07378c35a6994825a44b13925d4903d2-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/07378c35a6994825a44b13925d4903d2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 525 COLUMNS
At line 7148 RHS
At line 7669 BOUNDS
At line 8652 ENDATA
Problem MODEL has 520 rows, 982 columns and 3676 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 68.3857 - 0.00 seconds
Cgl0004I processed model has 518 rows, 982 columns (982 integer (982 of which binary)) and 3437 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.416667
Cbc0038I Solution found of -68.2821
Cbc0038I Before mini branch and bound, 980 integers at bound fixed and 0 continuous
Cbc0038I F

,position,name,team,price,projected_points_with_decay
245,GK,Jordan Pickford,Everton,4.5,3.914122
27,DEF,Andrew Robertson,Liverpool,6.7,4.672260
277,DEF,Layvin Kurzawa,Fulham,4.3,5.831671
309,DEF,Manuel Akanji,Manchester City,5.0,4.118100
232,MID,Joe Rothwell,Bournemouth,4.9,4.573684
264,MID,Kevin De Bruyne,Manchester City,12.6,6.870348
345,MID,Mohamed Salah,Liverpool,12.7,7.657413
53,MID,Bobby Clark,Liverpool,4.5,4.949845
14,FWD,Aleksandar Mitrovic,Fulham,6.8,5.293811
143,FWD,Erling Haaland,Manchester City,11.8,6.396338


In [16]:
my_df['dream_team_appearances'] = [sum(team.count(player_id) for team in results) for player_id in my_df.index]
my_df.head(5)

,name,team,position,price,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
0,Aaron Cresswell,West Ham,DEF,4.8,3.350040,3.350040,0.107745,0
1,Aaron Hickey,Brentford,DEF,4.9,1.735689,1.735689,0.098689,0
2,Aaron Ramsdale,Arsenal,GK,4.9,3.717104,3.717104,0.111077,0
3,Aaron Wan-Bissaka,Manchester Utd,DEF,4.3,2.932375,2.932375,0.202218,0
4,Abdoulaye Doucoure,Everton,MID,5.3,2.452793,2.452793,0.172072,0


In [17]:
print('Most appearances, all players:')
display(my_df.sort_values(by='dream_team_appearances', ascending=False).head(20))

Most appearances, all players:


,name,team,position,price,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
345,Mohamed Salah,Liverpool,MID,12.7,7.195047,7.195047,0.509939,100
166,Harry Kane,Tottenham,FWD,11.6,6.257554,6.257554,0.389150,93
323,Martin degaard,Arsenal,MID,6.4,4.873374,4.873374,0.165816,72
27,Andrew Robertson,Liverpool,DEF,6.7,4.669917,4.669917,0.237642,68
14,Aleksandar Mitrovic,Fulham,FWD,6.8,4.941288,4.941288,0.404834,67
341,Miguel Almiron,Newcastle Utd,MID,5.8,4.701961,4.701961,0.516686,59
143,Erling Haaland,Manchester City,FWD,11.8,5.632240,5.632240,1.182064,48
264,Kevin De Bruyne,Manchester City,MID,12.6,5.850493,5.850493,0.425962,47
21,Alisson,Liverpool,GK,5.4,4.213799,4.213799,0.181777,44
154,Gabriel Dos Santos,Arsenal,DEF,5.1,4.068051,4.068051,0.299264,41


In [18]:
print('GOALKEEPERS')
display(my_df[my_df['position']=='GK'].sort_values(by='dream_team_appearances', ascending=False).head(10))

GOALKEEPERS


,name,team,position,price,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
21,Alisson,Liverpool,GK,5.4,4.213799,4.213799,0.181777,44
131,Ederson,Manchester City,GK,5.4,3.989867,3.989867,0.180878,31
245,Jordan Pickford,Everton,GK,4.5,3.681400,3.681400,0.214038,13
111,Dean Henderson,Nottingham Forest,GK,4.7,3.736052,3.736052,0.218852,11
407,Robert Sanchez,Brighton,GK,4.6,3.518817,3.518817,0.122095,1
2,Aaron Ramsdale,Arsenal,GK,4.9,3.717104,3.717104,0.111077,0
258,Kasper Schmeichel,Leicester City,GK,5.0,2.910725,2.910725,0.118888,0
484,Vicente Guaita,Crystal Palace,GK,4.5,3.350147,3.350147,0.154744,0
472,Tom Heaton,Manchester Utd,GK,3.9,3.012849,3.012849,0.123339,0
410,Robin Olsen,Aston Villa,GK,3.9,2.273061,2.273061,0.137822,0


In [19]:
print('DEFENDERS')
display(my_df[my_df['position']=='DEF'].sort_values(by='dream_team_appearances', ascending=False).head(20))

DEFENDERS


,name,team,position,price,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
27,Andrew Robertson,Liverpool,DEF,6.7,4.669917,4.669917,0.237642,68
154,Gabriel Dos Santos,Arsenal,DEF,5.1,4.068051,4.068051,0.299264,41
476,Trent Alexander-Arnold,Liverpool,DEF,7.2,4.656335,4.656335,0.182254,38
135,Emil Krafth,Newcastle Utd,DEF,4.3,2.936405,2.936405,1.360462,25
275,Kyle Walker,Manchester City,DEF,4.8,3.932440,3.932440,0.207316,21
224,Joao Cancelo,Manchester City,DEF,7.4,4.386390,4.386390,0.194552,18
277,Layvin Kurzawa,Fulham,DEF,4.3,2.851572,2.851572,1.356024,15
158,Giulian Biancone,Nottingham Forest,DEF,4.2,3.027063,3.027063,0.961207,12
240,John Stones,Manchester City,DEF,5.4,3.952534,3.952534,0.185451,10
188,Jack Stacey,Bournemouth,DEF,4.3,3.070409,3.070409,0.988997,10


In [20]:
print('MIDFIELDERS')
display(my_df[my_df['position']=='MID'].sort_values(by='dream_team_appearances', ascending=False).head(20))

MIDFIELDERS


,name,team,position,price,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
345,Mohamed Salah,Liverpool,MID,12.7,7.195047,7.195047,0.509939,100
323,Martin degaard,Arsenal,MID,6.4,4.873374,4.873374,0.165816,72
341,Miguel Almiron,Newcastle Utd,MID,5.8,4.701961,4.701961,0.516686,59
264,Kevin De Bruyne,Manchester City,MID,12.6,5.850493,5.850493,0.425962,47
493,Wilfried Zaha,Crystal Palace,MID,7.5,4.787946,4.787946,0.295556,30
64,Bukayo Saka,Arsenal,MID,8.0,4.854097,4.854097,0.258678,25
279,Leandro Trossard,Brighton,MID,7.1,4.674847,4.674847,0.194351,17
173,Heung-Min Son,Tottenham,MID,11.6,5.252622,5.252622,0.315925,15
61,Bruno Fernandes,Manchester Utd,MID,9.8,4.828325,4.828325,0.552330,14
301,Luke Harris,Fulham,MID,4.4,2.835483,2.835483,1.339108,11


In [21]:
print('FORWARDS')
display(my_df[my_df['position']=='FWD'].sort_values(by='dream_team_appearances', ascending=False).head(20))

FORWARDS


,name,team,position,price,projected_points_per_game,projected_points_with_decay,std,dream_team_appearances
166,Harry Kane,Tottenham,FWD,11.6,6.257554,6.257554,0.389150,93
14,Aleksandar Mitrovic,Fulham,FWD,6.8,4.941288,4.941288,0.404834,67
143,Erling Haaland,Manchester City,FWD,11.8,5.632240,5.632240,1.182064,48
19,Alexander Isak,Newcastle Utd,FWD,6.6,4.483865,4.483865,0.567287,26
155,Gabriel Jesus,Arsenal,FWD,8.0,4.869891,4.869891,0.128544,11
107,Darwin Nunez,Liverpool,FWD,8.9,4.520697,4.520697,0.590963,8
373,Ollie Watkins,Aston Villa,FWD,7.1,3.419568,3.419568,0.320687,0
253,Kai Havertz,Chelsea,FWD,7.7,3.536595,3.536595,0.295230,0
260,Keinan Davis,Aston Villa,FWD,5.4,2.099473,2.099473,0.164402,0
261,Kelechi Iheanacho,Leicester City,FWD,6.1,2.414417,2.414417,0.132235,0
